In [1]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import tree
import warnings
warnings.filterwarnings('ignore')

In [4]:
#PATH = 'covid_19_in_education.xlsx'
df = pd.read_csv('covid_19_in_education.csv')

In [5]:
df.head()

,submission_id,submission_date,gender,age,geography,financial_situation,education,employment_status,submission_state,Unnamed: 9,can_children_observe_deterioration_of_basic_services_of_school,do_children_3_and_17_yrs_receive_regular_school_meals,are_there_teachers_at_scheduled_class_hours
0,4.570530e+15,17-03-2021,Female,Over 45 years old,Suburban/Peri-urban,"I can afford food and regular expenses, but no...",University or college degree completed,I am unemployed,Miranda,NaN,1,Every day,Irregularly
1,6.440170e+15,29-03-2021,Male,26 to 35 years old,Suburban/Peri-urban,I cannot afford enough food for my family,University or college degree completed,I am unemployed,Miranda,NaN,1,No,Irregularly
2,5.001170e+15,18-03-2021,Female,26 to 35 years old,City center or metropolitan area,"I can comfortably afford food, clothes, and fu...",University or college degree completed,"I work full-time, either as an employee or sel...",Miranda,NaN,1,No,There are not enough
3,5.524930e+15,24-03-2021,Male,36 to 45 years old,Suburban/Peri-urban,"I can afford food, but nothing else",University or college degree completed,"I work full-time, either as an employee or sel...",Miranda,NaN,1,No,There are enough
4,5.032520e+15,18-03-2021,Female,Over 45 years old,Suburban/Peri-urban,"I can afford food and regular expenses, but no...",University or college degree completed,I am unemployed,Miranda,NaN,1,No,There are not enough


deleting unnecessary columns

In [6]:
df = df.drop('submission_id', axis=1)
df = df.drop('submission_date', axis=1)
df = df.drop('Unnamed: 9', axis=1)

In [7]:
df.head()

,gender,age,geography,financial_situation,education,employment_status,submission_state,can_children_observe_deterioration_of_basic_services_of_school,do_children_3_and_17_yrs_receive_regular_school_meals,are_there_teachers_at_scheduled_class_hours
0,Female,Over 45 years old,Suburban/Peri-urban,"I can afford food and regular expenses, but no...",University or college degree completed,I am unemployed,Miranda,1,Every day,Irregularly
1,Male,26 to 35 years old,Suburban/Peri-urban,I cannot afford enough food for my family,University or college degree completed,I am unemployed,Miranda,1,No,Irregularly
2,Female,26 to 35 years old,City center or metropolitan area,"I can comfortably afford food, clothes, and fu...",University or college degree completed,"I work full-time, either as an employee or sel...",Miranda,1,No,There are not enough
3,Male,36 to 45 years old,Suburban/Peri-urban,"I can afford food, but nothing else",University or college degree completed,"I work full-time, either as an employee or sel...",Miranda,1,No,There are enough
4,Female,Over 45 years old,Suburban/Peri-urban,"I can afford food and regular expenses, but no...",University or college degree completed,I am unemployed,Miranda,1,No,There are not enough


finding unique values in "age" column

In [8]:
n = len(pd.unique(df['age']))
print("No.of.unique values :",n)

No.of.unique values : 6


In [9]:
a = df['age'].unique()
print(sorted(a))

['16 to 25 years old', '26 to 35 years old', '36 to 45 years old', 'Not Available', 'Over 45 years old', 'Under 16']


In [10]:
df.head()

,gender,age,geography,financial_situation,education,employment_status,submission_state,can_children_observe_deterioration_of_basic_services_of_school,do_children_3_and_17_yrs_receive_regular_school_meals,are_there_teachers_at_scheduled_class_hours
0,Female,Over 45 years old,Suburban/Peri-urban,"I can afford food and regular expenses, but no...",University or college degree completed,I am unemployed,Miranda,1,Every day,Irregularly
1,Male,26 to 35 years old,Suburban/Peri-urban,I cannot afford enough food for my family,University or college degree completed,I am unemployed,Miranda,1,No,Irregularly
2,Female,26 to 35 years old,City center or metropolitan area,"I can comfortably afford food, clothes, and fu...",University or college degree completed,"I work full-time, either as an employee or sel...",Miranda,1,No,There are not enough
3,Male,36 to 45 years old,Suburban/Peri-urban,"I can afford food, but nothing else",University or college degree completed,"I work full-time, either as an employee or sel...",Miranda,1,No,There are enough
4,Female,Over 45 years old,Suburban/Peri-urban,"I can afford food and regular expenses, but no...",University or college degree completed,I am unemployed,Miranda,1,No,There are not enough


In [11]:
#df['gender'] = int(df['gender'])
print (df.dtypes)

gender                                                            object
age                                                               object
geography                                                         object
financial_situation                                               object
education                                                         object
employment_status                                                 object
submission_state                                                  object
can_children_observe_deterioration_of_basic_services_of_school     int64
do_children_3_and_17_yrs_receive_regular_school_meals             object
are_there_teachers_at_scheduled_class_hours                       object
dtype: object


In [12]:
n = len(pd.unique(df['financial_situation']))
print("No.of.unique values :",n)

print()

a = df['financial_situation'].unique()
print(sorted(a))

No.of.unique values : 8

['I can afford food and regular expenses, but nothing else', 'I can afford food, but nothing else', 'I can afford food, regular expenses, and clothes, but nothing else', 'I can comfortably afford food, clothes, and furniture, and I have savings', 'I can comfortably afford food, clothes, and furniture, but I don’t have savings', 'I cannot afford enough food for my family', 'Not Available', 'Prefer not to answer']


In [13]:
n = len(pd.unique(df['gender']))
print("No.of.unique values :",n)

print()

a = df['gender'].unique()
print(sorted(a))

No.of.unique values : 5

['Female', 'Male', 'Non-Binary', 'Not Available', 'Prefer not to answer']


In [14]:
df = df.drop('can_children_observe_deterioration_of_basic_services_of_school', axis=1)
df = df.drop('do_children_3_and_17_yrs_receive_regular_school_meals', axis=1)
df = df.drop('are_there_teachers_at_scheduled_class_hours', axis=1)


In [15]:
print (df.dtypes)

gender                 object
age                    object
geography              object
financial_situation    object
education              object
employment_status      object
submission_state       object
dtype: object


In [16]:
final_data = pd.get_dummies(df, drop_first = True)

In [17]:
final_data.head()

,gender_Male,gender_Non-Binary,gender_Not Available,gender_Prefer not to answer,age_26 to 35 years old,age_36 to 45 years old,age_Not Available,age_Over 45 years old,age_Under 16,geography_Not Available,...,submission_state_Monagas,submission_state_Mérida,submission_state_Nueva Esparta,submission_state_Portuguesa,submission_state_Sucre,submission_state_Trujillo,submission_state_Táchira,submission_state_Vargas,submission_state_Yaracuy,submission_state_Zulia
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
n = len(pd.unique(df['submission_state']))
print("No.of.unique values :",n)


No.of.unique values : 24
